# Projeto Avaliativo 02

> [Universidade Federal do Ceará (UFC)](https://www.ufc.br/)\
> [Departamento de Computação (DC)](https://dc.ufc.br/pt/)\
> [Capacitação Técnica e Empreendedora em IA (CTE-IA)](https://www.cteia.dc.ufc.br/)\
> Fase I: Capacitação Teórica de IA\
> Disciplina: Processamento de Linguagem Natural (45h)\
> Professor: [Ticiana Linhares Coelho da Silva](http://lattes.cnpq.br/3125027229507836)\
> E-mail: <ticianalc@virtual.ufc.br>\
> Discente: Lucas Cavalcante dos Santos

## Instruções para Execução do Projeto Avaliativo

Este trabalho tem como objetivo avaliar o desempenho de modelos de Question Answering (QA) disponíveis no Hugging Face, utilizando um subconjunto do dataset BeIR / DBpedia Entity Generated Queries (https://huggingface.co/datasets/BeIR/dbpedia-entity-generated-queries). A proposta vai além da simples execução de modelos, exigindo análise quantitativa e qualitativa das respostas geradas, bem como reflexão sobre a viabilidade de uso desses modelos em cenários reais.

### 01. Seleção do Subconjunto de Dados

- Cada aluno deverá selecionar um intervalo exclusivo de 1000 exemplos, que será disponibilizado via um arquivo csv.
- Não é permitida sobreposição entre os intervalos escolhidos pelos alunos, ou seja, dois alunos não podem escolher o mesmo arquivo.
- A escolha dos intervalos será organizada através da planilha, previamente entre os alunos (mediada pelos professores e monitora), garantindo exclusividade.
- Para realizar a escolha, entre no documento "2ª Avaliação - Escolha dos Intervalos" e siga as instruções presentes lá. (Link: https://docs.google.com/document/d/1tT_X0aRS8Bc2RcmkvhdS-Yerj2Z2_cm0DHhmDwidwVE/edit?usp=sharing). Cada exemplo contém:
  - Contexto (text): trecho textual curto na coluna text; e
  - Pergunta (query): questão a ser respondida com base no contexto.

### 02. Seleção dos Modelos

- Cada aluno deverá escolher dois modelos distintos de Question Answering disponíveis no Hugging Face.
- Os modelos devem ser executados sobre o mesmo subconjunto de dados (1.000 exemplos).

### 03. Critérios de Avaliação

A comparação entre os dois modelos deverá considerar obrigatoriamente os seguintes critérios:

* A) Tamanho médio das perguntas
  - Calcular o tamanho médio das perguntas (query) dos 1000 exemplos;
* B) Score médio das respostas
  - Calcular o score médio retornado pelo modelo (na resposta) ao longo dos 1000 exemplos;
  - Analisar se o score médio reflete, de forma consistente, a qualidade das respostas geradas.
* C) Overlap entre contexto e resposta
  - Calcular o overlap médio entre as palavras da resposta gerada e o contexto correspondente;
  - Considerar que:
    - Alta sobreposição tende a indicar que a resposta está explícita no contexto;
    - Baixa sobreposição pode indicar inferência incorreta ou potencial alucinação.
  - Comparar os dois modelos quanto a esse comportamento.
* D) Avaliação qualitativa por amostragem guiada
  - Cada aluno deverá realizar uma análise manual e qualitativa de 25 exemplos, distribuídos da seguinte forma:
    - 10 exemplos com maior score;
    - 10 exemplos com menor score;
    - 5 mesmos exemplos em que os dois modelos produziram respostas diferentes e que não são os 10 melhores e 10 piores scores.
  - Para esses exemplos, o aluno deverá analisar:
    - Se a resposta está de fato presente no contexto;
    - Se a resposta está correta ou parcialmente correta;
    - Se há evidência de alucinação.
  - Com base nessa análise, o aluno deverá responder obrigatoriamente à seguinte questão:

“Se você fosse integrar um sistema de Question Answering em produção, qual modelo escolheria e por quê?”

A resposta deve ser explicitamente fundamentada nos resultados da avaliação qualitativa (item D), e não apenas em métricas automáticas.

### Tripla escolhida abaixo:

- Arquivo: shard_055.csv
- Modelo 1: [distilbert/distilbert-base-cased-distilled-squad](https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad)
- Modelo 2: [deepset/roberta-base-squad2](https://huggingface.co/deepset/roberta-base-squad2)

## Pipeline

### Etapas Preliminares

#### Instalação das Bibliotecas Essenciais

In [1]:
# Importação de bibliotecas necessárias

import os
import pandas as pd
import re
import torch
import zipfile
from google.colab import files
from transformers import pipeline
import warnings
warnings.filterwarnings('ignore')

#### Configurações Globais

In [2]:
# ========== CONFIGURAÇÕES E CONSTANTES ==========
DATASET_PATH = './datasets/Intervalos/shard_055.csv'
MODEL_CONFIGS = {
    'distilbert': {
        'model_name': 'distilbert-base-cased-distilled-squad',
        'tokenizer_name': 'distilbert-base-cased-distilled-squad',
        'fallback_model': 'distilbert-base-uncased-distilled-squad'
    },
    'roberta': {
        'model_name': 'deepset/roberta-base-squad2',
        'tokenizer_name': 'deepset/roberta-base-squad2',
        'fallback_model': 'deepset/bert-base-cased-squad2'
    }
}
EXPORT_DIR = 'export_qa_analysis'
CONTEXT_LIMIT = 512  # Limite de caracteres para o contexto

# Verificar se GPU está disponível
device = 0 if torch.cuda.is_available() else -1
gpu_name = torch.cuda.get_device_name(0) if torch.cuda.is_available() else "N/A"
print(f"🚀 Dispositivo de execução: {'GPU' if device == 0 else 'CPU'}")
if device == 0:
    print(f"   GPU: {gpu_name}")
    print(f"   Memória GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Configurações de performance baseadas no dispositivo
if device == 0:
    BATCH_SIZE = 32
    print(f"   Batch size configurado: {BATCH_SIZE} (GPU)")
else:
    BATCH_SIZE = 16
    print(f"   Batch size configurado: {BATCH_SIZE} (CPU)")

🚀 Dispositivo de execução: CPU
   Batch size configurado: 16 (CPU)


#### Importação do Conjunto de Dados

In [3]:
def load_dataset():
    """Carrega o dataset do arquivo RAR"""
    # Verificar se já existe extraído
    if os.path.exists(DATASET_PATH):
        print("✅ Dataset já existe localmente.")
        return pd.read_csv(DATASET_PATH)

    print("📤 Por favor, faça upload do arquivo RAR (Intervalos.rar):")
    uploaded = files.upload()

    # Encontrar arquivo RAR
    rar_filename = None
    for filename in uploaded.keys():
        if filename.endswith('.rar'):
            rar_filename = filename
            break

    if not rar_filename:
        raise ValueError("❌ Nenhum arquivo RAR encontrado no upload.")

    # Instalar rarfile se necessário
    try:
        import rarfile
    except ImportError:
        print("📦 Instalando rarfile...")
        !pip install rarfile
        import rarfile

    # Extrair RAR
    with rarfile.RarFile(rar_filename, 'r') as rar_ref:
        rar_ref.extractall('./datasets')

    print("✅ Arquivo RAR extraído com sucesso!")

    # Carregar CSV
    return pd.read_csv(DATASET_PATH)

# Carregar o dataset
print("📂 Iniciando carregamento do dataset...")
df = load_dataset()
print(f"✅ Dataset carregado! Shape: {df.shape}")
print(f"📊 Colunas: {list(df.columns)}")

📂 Iniciando carregamento do dataset...
📤 Por favor, faça upload do arquivo RAR (Intervalos.rar):


Saving Intervalos.rar to Intervalos.rar
📦 Instalando rarfile...
✅ Arquivo RAR extraído com sucesso!
✅ Dataset carregado! Shape: (1000, 4)
📊 Colunas: ['_id', 'title', 'text', 'query']


#### Análise Exploratória Simples

In [4]:
def display_dataframe_statistics(df):
    """Exibe estatísticas do DataFrame"""
    print(f"\n📊 ESTATÍSTICAS DO DATASET:")
    print("="*60)
    print(f"• Total de exemplos: {len(df)}")
    print(f"• Colunas disponíveis: {list(df.columns)}")
    print("="*60)

# Exibir estatísticas
display_dataframe_statistics(df)

# Exibir primeiras linhas
print("\n📋 PRIMEIRAS 5 LINHAS DO DATASET:")
print("="*60)
display(df.head())


📊 ESTATÍSTICAS DO DATASET:
• Total de exemplos: 1000
• Colunas disponíveis: ['_id', 'title', 'text', 'query']

📋 PRIMEIRAS 5 LINHAS DO DATASET:


,_id,title,text,query
0,"<dbpedia:Mars_Hill-Blaine,_Maine>","Mars Hill-Blaine, Maine",Mars Hill-Blaine was a census-designated place...,mars hill malaine me population
1,"<dbpedia:Winnfield,_Louisiana>","Winnfield, Louisiana",Winnfield is a small city in the parish seat o...,what parish is winnfield louisiana
2,"<dbpedia:Caribou,_Maine>","Caribou, Maine",Caribou is the second largest city in Aroostoo...,what county is cartibou maine in?
3,"<dbpedia:Littleton,_Maine>","Littleton, Maine","Littleton is a town in Aroostook County, Maine...","what county is littleton, me in"
4,"<dbpedia:Hammond,_Maine>","Hammond, Maine","Hammond is a town in Aroostook County, Maine, ...",what county is hammond maine in


#### Preparação do Dataset

In [5]:
def prepare_dataframe(df):
    """Prepara o DataFrame renomeando colunas e calculando estatísticas"""
    # Renomear colunas para facilitar o uso
    if 'text' in df.columns and 'query' in df.columns:
        df = df.rename(columns={'text': 'context', 'query': 'question'})
        print("✓ Colunas renomeadas: 'text' → 'context', 'query' → 'question'")
    elif 'context' not in df.columns or 'question' not in df.columns:
        print("⚠️  Atenção: Verificando estrutura das colunas...")
        print(f"📋 Colunas atuais: {list(df.columns)}")
        # Tentar identificar colunas automaticamente
        for col in df.columns:
            if 'text' in col.lower() or 'context' in col.lower():
                df = df.rename(columns={col: 'context'})
            elif 'query' in col.lower() or 'question' in col.lower():
                df = df.rename(columns={col: 'question'})
        print(f"📋 Colunas após ajuste: {list(df.columns)}")

    # Verificar se temos as colunas necessárias
    required_columns = ['context', 'question']
    if not all(col in df.columns for col in required_columns):
        print("❌ Erro: Colunas 'context' e 'question' não encontradas.")
        print("As colunas disponíveis são:", list(df.columns))
        raise ValueError("O dataset deve conter colunas 'context' e 'question' (ou equivalentes).")

    # Calcular estatísticas do texto
    df['context_length'] = df['context'].apply(lambda x: len(str(x).split()))
    df['question_length'] = df['question'].apply(lambda x: len(str(x).split()))

    return df

# Preparar o dataframe
print("🔧 Preparando dataframe...")
df = prepare_dataframe(df)
print("✅ Dataframe preparado!")

def display_dataframe_statistics(df):
    """Exibe estatísticas do DataFrame"""
    print(f"\n📊 ESTATÍSTICAS DO DATASET:")
    print("="*60)
    print(f"• Total de exemplos: {len(df)}")
    print(f"• Colunas disponíveis: {list(df.columns)}")
    print(f"• Contexto médio: {df['context_length'].mean():.1f} palavras")
    print(f"• Pergunta média: {df['question_length'].mean():.1f} palavras")
    print(f"• Contexto máximo: {df['context_length'].max()} palavras")
    print(f"• Pergunta máxima: {df['question_length'].max()} palavras")
    print("="*60)

# Exibir estatísticas
display_dataframe_statistics(df)

# Exibir primeiras linhas
print("\n📋 PRIMEIRAS 5 LINHAS DO DATASET:")
print("="*60)
display(df.head())

# Usar dataset completo para processamento
df_sample = df.copy()
print(f"\n🎯 Dataset completo será processado: {len(df_sample)} exemplos")

🔧 Preparando dataframe...
✓ Colunas renomeadas: 'text' → 'context', 'query' → 'question'
✅ Dataframe preparado!

📊 ESTATÍSTICAS DO DATASET:
• Total de exemplos: 1000
• Colunas disponíveis: ['_id', 'title', 'context', 'question', 'context_length', 'question_length']
• Contexto médio: 40.0 palavras
• Pergunta média: 5.3 palavras
• Contexto máximo: 115 palavras
• Pergunta máxima: 9 palavras

📋 PRIMEIRAS 5 LINHAS DO DATASET:


,_id,title,context,question,context_length,question_length
0,"<dbpedia:Mars_Hill-Blaine,_Maine>","Mars Hill-Blaine, Maine",Mars Hill-Blaine was a census-designated place...,mars hill malaine me population,47,5
1,"<dbpedia:Winnfield,_Louisiana>","Winnfield, Louisiana",Winnfield is a small city in the parish seat o...,what parish is winnfield louisiana,39,5
2,"<dbpedia:Caribou,_Maine>","Caribou, Maine",Caribou is the second largest city in Aroostoo...,what county is cartibou maine in?,20,6
3,"<dbpedia:Littleton,_Maine>","Littleton, Maine","Littleton is a town in Aroostook County, Maine...","what county is littleton, me in",60,6
4,"<dbpedia:Hammond,_Maine>","Hammond, Maine","Hammond is a town in Aroostook County, Maine, ...",what county is hammond maine in,18,6



🎯 Dataset completo será processado: 1000 exemplos


### Implementação dos Modelos

In [6]:
# ========== FUNÇÕES DE PROCESSAMENTO DE TEXTO ==========
def clean_text(text):
    """Limpa o texto removendo caracteres especiais e normalizando"""
    if not isinstance(text, str):
        return ""
    # Remover caracteres especiais, manter letras, números e espaços
    text = re.sub(r'[\w\s\.\,\-\\?]', ' ', text)
    # Remover múltiplos espaços
    text = re.sub(r'\s+', ' ', text)
    return text.strip().lower()

def calculate_overlap(context, answer):
    """Calcula a sobreposição de palavras entre contexto e resposta"""
    if not answer or not context:
        return 0

    context_words = set(clean_text(context).split())
    answer_words = set(clean_text(answer).split())

    if not answer_words:
        return 0

    # Calcular Jaccard similarity
    intersection = len(context_words.intersection(answer_words))
    union = len(context_words.union(answer_words))

    return intersection / len(answer_words) if len(answer_words) > 0 else 0

# ========== FUNÇÕES DE MODELO E INFERÊNCIA ==========
def load_qa_model(model_key, device):
    """Carrega um modelo de QA específico"""
    config = MODEL_CONFIGS[model_key]

    print(f"🔧 Carregando modelo {model_key}: {config['model_name']}...")
    try:
        qa_pipeline = pipeline(
            "question-answering",
            model=config['model_name'],
            tokenizer=config['tokenizer_name'],
            device=device
        )
        print(f"✅ Modelo {model_key} carregado com sucesso!")
    except Exception as e:
        print(f"❌ Erro ao carregar {model_key}: {e}")
        # Fallback para modelo alternativo
        print(f"🔄 Tentando carregar modelo alternativo: {config['fallback_model']}")
        qa_pipeline = pipeline(
            "question-answering",
            model=config['fallback_model'],
            device=device
        )

    return qa_pipeline

def get_qa_result(pipeline_model, question, context, idx):
    """Executa o modelo de QA e retorna resultados formatados"""
    try:
        result = pipeline_model({
            'question': question,
            'context': context[:CONTEXT_LIMIT]  # Limitar contexto para evitar overflow
        })

        return {
            'answer': result['answer'],
            'score': result['score'],
            'start': result['start'],
            'end': result['end']
        }
    except Exception as e:
        print(f"⚠️  Erro no exemplo {idx}: {e}")
        return {
            'answer': '',
            'score': 0.0,
            'start': 0,
            'end': 0
        }

# ========== CARREGAR MODELOS ==========
print("\n" + "="*60)
print("🤖 CARREGAMENTO DOS MODELOS")
print("="*60)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔧 Dispositivo configurado: {device}")

# Carregar modelos
qa_pipeline_distilbert = load_qa_model('distilbert', device)
qa_pipeline_roberta = load_qa_model('roberta', device)

# ========== PROCESSAR DATASET COM MODELOS ==========
print("\n" + "="*60)
print("⚙️  PROCESSAMENTO DOS MODELOS")
print("="*60)

def process_dataset_with_models(df_sample, qa_pipeline_distilbert, qa_pipeline_roberta, batch_size=50):
    """Processa o dataset com ambos os modelos"""
    results_distilbert = []
    results_roberta = []
    overlaps_distilbert = []
    overlaps_roberta = []

    total_samples = len(df_sample)

    for idx, row in df_sample.iterrows():
        if idx % batch_size == 0:
            print(f"📊 Processando exemplo {idx+1}/{total_samples}...")

            # Limpar cache da GPU se estiver usando GPU
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

        question = str(row['question'])
        context = str(row['context'])

        # Executar modelo DistilBERT
        distilbert_result = get_qa_result(qa_pipeline_distilbert, question, context, idx)
        results_distilbert.append(distilbert_result)

        # Executar modelo RoBERTa
        roberta_result = get_qa_result(qa_pipeline_roberta, question, context, idx)
        results_roberta.append(roberta_result)

        # Calcular overlaps
        overlap_dist = calculate_overlap(context, distilbert_result['answer'])
        overlap_rob = calculate_overlap(context, roberta_result['answer'])

        overlaps_distilbert.append(overlap_dist)
        overlaps_roberta.append(overlap_rob)

    return results_distilbert, results_roberta, overlaps_distilbert, overlaps_roberta

# Processar dataset completo
print(f"\n🎯 Processando dataset completo com {len(df_sample)} exemplos...")
print(f"⚠️  Atenção: Processamento completo pode demorar dependendo do tamanho do dataset")

# Definir batch size baseado no tamanho do dataset
if len(df_sample) > 10000:
    batch_report_size = 100  # Reportar progresso a cada 100 exemplos para datasets grandes
elif len(df_sample) > 5000:
    batch_report_size = 50
else:
    batch_report_size = 25

results = process_dataset_with_models(df_sample, qa_pipeline_distilbert, qa_pipeline_roberta, batch_report_size)
results_distilbert, results_roberta, overlaps_distilbert, overlaps_roberta = results

print("✅ Processamento concluído!")

# ========== ADICIONAR RESULTADOS AO DATAFRAME PRINCIPAL ==========
print("\n" + "="*60)
print("📊 ADICIONANDO RESULTADOS AO DATAFRAME PRINCIPAL")
print("="*60)

# Adicionar resultados diretamente ao df_sample
df_sample['distilbert_answer'] = [r['answer'] for r in results_distilbert]
df_sample['distilbert_score'] = [r['score'] for r in results_distilbert]
df_sample['roberta_answer'] = [r['answer'] for r in results_roberta]
df_sample['roberta_score'] = [r['score'] for r in results_roberta]
df_sample['overlap_distilbert'] = overlaps_distilbert
df_sample['overlap_roberta'] = overlaps_roberta

# Calcular diferenças entre modelos
df_sample['score_difference'] = df_sample['roberta_score'] - df_sample['distilbert_score']
df_sample['overlap_difference'] = df_sample['overlap_roberta'] - df_sample['overlap_distilbert']

# Determinar qual modelo teve melhor score por questão
df_sample['melhor_modelo_score'] = df_sample.apply(
    lambda x: 'DistilBERT' if x['distilbert_score'] > x['roberta_score'] else
              ('RoBERTa' if x['roberta_score'] > x['distilbert_score'] else 'Empate'),
    axis=1
)

# Determinar qual modelo teve maior overlap por questão
df_sample['melhor_modelo_overlap'] = df_sample.apply(
    lambda x: 'DistilBERT' if x['overlap_distilbert'] > x['overlap_roberta'] else
              ('RoBERTa' if x['overlap_roberta'] > x['overlap_distilbert'] else 'Empate'),
    axis=1
)

# Calcular melhor e pior score global
df_sample['melhor_score_global'] = df_sample[['distilbert_score', 'roberta_score']].max(axis=1)
df_sample['pior_score_global'] = df_sample[['distilbert_score', 'roberta_score']].min(axis=1)
df_sample['modelo_melhor_score'] = df_sample.apply(
    lambda x: 'DistilBERT' if x['distilbert_score'] == x['melhor_score_global'] else 'RoBERTa',
    axis=1
)
df_sample['modelo_pior_score'] = df_sample.apply(
    lambda x: 'DistilBERT' if x['distilbert_score'] == x['pior_score_global'] else 'RoBERTa',
    axis=1
)

print(f"✅ DataFrame principal atualizado! Total de colunas: {len(df_sample.columns)}")
print(f"📋 Colunas disponíveis: {list(df_sample.columns)}")


🤖 CARREGAMENTO DOS MODELOS
🔧 Dispositivo configurado: cpu
🔧 Carregando modelo distilbert: distilbert-base-cased-distilled-squad...


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu


✅ Modelo roberta carregado com sucesso!

⚙️  PROCESSAMENTO DOS MODELOS

🎯 Processando dataset completo com 1000 exemplos...
⚠️  Atenção: Processamento completo pode demorar dependendo do tamanho do dataset
📊 Processando exemplo 1/1000...
📊 Processando exemplo 26/1000...
📊 Processando exemplo 51/1000...
📊 Processando exemplo 76/1000...
📊 Processando exemplo 101/1000...
📊 Processando exemplo 126/1000...
📊 Processando exemplo 151/1000...
📊 Processando exemplo 176/1000...
📊 Processando exemplo 201/1000...
📊 Processando exemplo 226/1000...
📊 Processando exemplo 251/1000...
📊 Processando exemplo 276/1000...
📊 Processando exemplo 301/1000...
📊 Processando exemplo 326/1000...
📊 Processando exemplo 351/1000...
📊 Processando exemplo 376/1000...
📊 Processando exemplo 401/1000...
📊 Processando exemplo 426/1000...
📊 Processando exemplo 451/1000...
📊 Processando exemplo 476/1000...
📊 Processando exemplo 501/1000...
📊 Processando exemplo 526/1000...
📊 Processando exemplo 551/1000...
📊 Processando exe

In [7]:
display(df_sample.head(5))

,_id,title,context,question,context_length,question_length,distilbert_answer,distilbert_score,roberta_answer,roberta_score,overlap_distilbert,overlap_roberta,score_difference,overlap_difference,melhor_modelo_score,melhor_modelo_overlap,melhor_score_global,pior_score_global,modelo_melhor_score,modelo_pior_score
0,"<dbpedia:Mars_Hill-Blaine,_Maine>","Mars Hill-Blaine, Maine",Mars Hill-Blaine was a census-designated place...,mars hill malaine me population,47,5,"1,428",0.363840,"1,428",0.616583,0.0,0.0,0.252743,0.0,RoBERTa,Empate,0.616583,0.363840,RoBERTa,DistilBERT
1,"<dbpedia:Winnfield,_Louisiana>","Winnfield, Louisiana",Winnfield is a small city in the parish seat o...,what parish is winnfield louisiana,39,5,Winn Parish,0.435036,Winn Parish,0.851112,0.0,0.0,0.416076,0.0,RoBERTa,Empate,0.851112,0.435036,RoBERTa,DistilBERT
2,"<dbpedia:Caribou,_Maine>","Caribou, Maine",Caribou is the second largest city in Aroostoo...,what county is cartibou maine in?,20,6,Aroostook County,0.701953,Aroostook County,0.756754,0.0,0.0,0.054801,0.0,RoBERTa,Empate,0.756754,0.701953,RoBERTa,DistilBERT
3,"<dbpedia:Littleton,_Maine>","Littleton, Maine","Littleton is a town in Aroostook County, Maine...","what county is littleton, me in",60,6,Aroostook County,0.745770,Aroostook County,0.664746,0.0,0.0,-0.081024,0.0,DistilBERT,Empate,0.745770,0.664746,DistilBERT,RoBERTa
4,"<dbpedia:Hammond,_Maine>","Hammond, Maine","Hammond is a town in Aroostook County, Maine, ...",what county is hammond maine in,18,6,Aroostook County,0.726418,Aroostook County,0.748841,0.0,0.0,0.022423,0.0,RoBERTa,Empate,0.748841,0.726418,RoBERTa,DistilBERT


### Exportação dos Resultados

In [8]:
# ========== EXPORTAÇÃO E DOWNLOAD AUTOMÁTICO ==========
print("\n" + "="*60)
print("💾 EXPORTAÇÃO E DOWNLOAD AUTOMÁTICO")
print("="*60)

import os
import zipfile
from datetime import datetime
from google.colab import files

# Criar diretório de exportação se não existir
EXPORT_DIR = 'export_results'
os.makedirs(EXPORT_DIR, exist_ok=True)

# Criar timestamp para nome do arquivo
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# 1. Exportar DataFrame completo (df_sample)
print("📊 Exportando DataFrame...")
df_export_path = os.path.join(EXPORT_DIR, f'qa_analysis_{timestamp}.csv')
df_sample.to_csv(df_export_path, index=False, encoding='utf-8')
print(f"✅ DataFrame exportado: {df_export_path}")
print(f"   • {len(df_sample)} exemplos, {len(df_sample.columns)} colunas")

# 2. Exportar arquivo de metadados
print("📝 Exportando metadados...")
metadata_path = os.path.join(EXPORT_DIR, f'metadata_{timestamp}.txt')

with open(metadata_path, 'w', encoding='utf-8') as f:
    f.write(f"METADADOS DA ANÁLISE DE QUESTION ANSWERING\n")
    f.write(f"="*60 + "\n")
    f.write(f"Data: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"\nESTATÍSTICAS:\n")
    f.write(f"-"*40 + "\n")
    f.write(f"Total: {len(df_sample)} exemplos\n")
    f.write(f"Score médio DistilBERT: {df_sample['distilbert_score'].mean():.4f}\n")
    f.write(f"Score médio RoBERTa: {df_sample['roberta_score'].mean():.4f}\n")
    f.write(f"Overlap médio DistilBERT: {df_sample['overlap_distilbert'].mean():.2%}\n")
    f.write(f"Overlap médio RoBERTa: {df_sample['overlap_roberta'].mean():.2%}\n")
    f.write(f"DistilBERT melhor score: {(df_sample['melhor_modelo_score'] == 'DistilBERT').sum()}\n")
    f.write(f"RoBERTa melhor score: {(df_sample['melhor_modelo_score'] == 'RoBERTa').sum()}\n")
    f.write(f"Empates: {(df_sample['melhor_modelo_score'] == 'Empate').sum()}\n")

print(f"✅ Metadados exportados: {metadata_path}")

# 3. Comprimir em ZIP
print("\n📦 Comprimindo arquivos...")
zip_filename = f'qa_results_{timestamp}.zip'

with zipfile.ZipFile(zip_filename, 'w') as zipf:
    # Adicionar CSV com nome simplificado
    zipf.write(df_export_path, arcname='qa_analysis.csv')
    # Adicionar metadados
    zipf.write(metadata_path, arcname='metadata.txt')

zip_size = os.path.getsize(zip_filename) / 1024
print(f"✅ Arquivo ZIP criado: {zip_filename}")
print(f"   • Tamanho: {zip_size:.1f} KB")

# 4. Baixar automaticamente
print("\n⬇️  Iniciando download...")
files.download(zip_filename)

print("\n" + "="*60)
print("✅ DOWNLOAD CONCLUÍDO!")
print("="*60)
print(f"📁 Arquivo: {zip_filename}")
print(f"📊 Conteúdo:")
print(f"   1. qa_analysis.csv - DataFrame completo com {len(df_sample)} exemplos")
print(f"   2. metadata.txt - Resumo estatístico da análise")
print("\n📍 Os arquivos também estão disponíveis localmente em:")
print(f"   • CSV: {os.path.abspath(df_export_path)}")
print(f"   • Metadados: {os.path.abspath(metadata_path)}")
print(f"   • ZIP: {os.path.abspath(zip_filename)}")


💾 EXPORTAÇÃO E DOWNLOAD AUTOMÁTICO
📊 Exportando DataFrame...
✅ DataFrame exportado: export_results/qa_analysis_20260202_130042.csv
   • 1000 exemplos, 20 colunas
📝 Exportando metadados...
✅ Metadados exportados: export_results/metadata_20260202_130042.txt

📦 Comprimindo arquivos...
✅ Arquivo ZIP criado: qa_results_20260202_130042.zip
   • Tamanho: 519.3 KB

⬇️  Iniciando download...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ DOWNLOAD CONCLUÍDO!
📁 Arquivo: qa_results_20260202_130042.zip
📊 Conteúdo:
   1. qa_analysis.csv - DataFrame completo com 1000 exemplos
   2. metadata.txt - Resumo estatístico da análise

📍 Os arquivos também estão disponíveis localmente em:
   • CSV: /content/export_results/qa_analysis_20260202_130042.csv
   • Metadados: /content/export_results/metadata_20260202_130042.txt
   • ZIP: /content/qa_results_20260202_130042.zip


### Métricas

A comparação entre os dois modelos deverá considerar obrigatoriamente os seguintes critérios:

* **A) Tamanho médio das perguntas:**
Calcular o tamanho médio das perguntas (query) dos 1.000 exemplos.

In [9]:
# Calcular a Média da Perguntas

media_p = df_sample['question_length'].mean()
print(f"Média do comprimento das perguntas: {media_p:.1f} palavras")

Média do comprimento das perguntas: 5.3 palavras


* **B) Score médio das respostas:**
Calcular o score médio retornado pelo modelo (na resposta) ao longo dos 1.000 exemplos.
Analisar se o score médio reflete, de forma consistente, a qualidade das respostas geradas.

In [10]:
# Calcular a Média dos Scores dos Modelos

media_s = df_sample[['distilbert_score', 'roberta_score']].mean()
print(f"Médias dos Scores dos Modelos:\n{media_s}")

Médias dos Scores dos Modelos:
distilbert_score    0.589144
roberta_score       0.570655
dtype: float64


Os scores médios calculados são: DistilBERT com 0.5891 e RoBERTa com 0.5707. Para analisar se esses scores refletem de forma consistente a qualidade das respostas geradas, como solicitado no critério B, é preciso realizar uma avaliação qualitativa. Você pode fazer isso revisando manualmente os exemplos nos DataFrames que foram gerados anteriormente, como:

distilbert_top e distilbert_bottom (melhores e piores do DistilBERT)
roberta_top e roberta_bottom (melhores e piores do RoBERTa)
global_top_melhores e global_top_piores (melhores e piores globais)
disagreement_df (exemplos onde os modelos discordam)
Ao inspecionar as respostas e seus respectivos scores nesses DataFrames, você poderá formar uma opinião sobre a consistência entre o score numérico e a qualidade percebida da resposta. Por exemplo, observe se as respostas com scores altos realmente são de alta qualidade e se as respostas com scores baixos são consistentemente ruins.

* **C) Overlap entre contexto e resposta:**
Calcular o overlap médio entre as palavras da resposta gerada e o contexto correspondente.
Considerar que:
Alta sobreposição tende a indicar que a resposta está explícita no contexto.
Baixa sobreposição pode indicar inferência incorreta ou potencial alucinação.

In [11]:
# Calcular a Média dos Overlaps dos Modelos

media_o = df_sample[['overlap_distilbert', 'overlap_roberta']].mean()
print(f"Médias dos Overlaps dos Modelos:\n{media_o}")

Médias dos Overlaps dos Modelos:
overlap_distilbert    0.079
overlap_roberta       0.079
dtype: float64


In [12]:
# ========== CATEGORIZAÇÃO DOS OVERLAPS ==========
print("\n" + "="*60)
print("📊 CATEGORIZAÇÃO DOS OVERLAPS")
print("="*60)

# Definir categorias baseadas na interpretação
def categorizar_overlap(overlap_value):
    """Categoriza o overlap baseado no valor"""
    if overlap_value >= 0.8:
        return "Muito Alto (Resposta explícita no contexto)"
    elif overlap_value >= 0.6:
        return "Alto (Boa correspondência)"
    elif overlap_value >= 0.4:
        return "Moderado (Correspondência parcial)"
    elif overlap_value >= 0.2:
        return "Baixo (Possível inferência)"
    elif overlap_value > 0:
        return "Muito Baixo (Risco de alucinação)"
    else:
        return "Sem overlap"

# Aplicar categorização para ambos os modelos
df_sample['categoria_overlap_distilbert'] = df_sample['overlap_distilbert'].apply(categorizar_overlap)
df_sample['categoria_overlap_roberta'] = df_sample['overlap_roberta'].apply(categorizar_overlap)

# Calcular distribuição das categorias
print("\n📈 DISTRIBUIÇÃO DAS CATEGORIAS DE OVERLAP:")
print("-"*50)

print("\n🤖 DISTILBERT:")
categorias_dist = df_sample['categoria_overlap_distilbert'].value_counts().sort_index()
for categoria, count in categorias_dist.items():
    percentual = (count / len(df_sample)) * 100
    print(f"  {categoria:<45} : {count:>4} exemplos ({percentual:>5.1f}%)")

print("\n🤖 ROBERTA:")
categorias_rob = df_sample['categoria_overlap_roberta'].value_counts().sort_index()
for categoria, count in categorias_rob.items():
    percentual = (count / len(df_sample)) * 100
    print(f"  {categoria:<45} : {count:>4} exemplos ({percentual:>5.1f}%)")

# Análise comparativa
print("\n📊 COMPARAÇÃO ENTRE MODELOS:")
print("-"*50)

# Calcular estatísticas por categoria
categorias = [
    "Muito Alto (Resposta explícita no contexto)",
    "Alto (Boa correspondência)",
    "Moderado (Correspondência parcial)",
    "Baixo (Possível inferência)",
    "Muito Baixo (Risco de alucinação)",
    "Sem overlap"
]

print(f"{'Categoria':<50} {'DistilBERT':>12} {'RoBERTa':>12} {'Diferença':>12}")
print("-"*86)

for categoria in categorias:
    count_dist = categorias_dist.get(categoria, 0)
    count_rob = categorias_rob.get(categoria, 0)
    percent_dist = (count_dist / len(df_sample)) * 100
    percent_rob = (count_rob / len(df_sample)) * 100
    dif_percent = percent_rob - percent_dist

    print(f"{categoria:<50} {percent_dist:>10.1f}% {percent_rob:>10.1f}% {dif_percent:>10.1f}pp")

# Análise de risco de alucinação
print("\n⚠️  ANÁLISE DE RISCO DE ALUCINAÇÃO:")
print("-"*50)

categorias_risco = [
    "Muito Baixo (Risco de alucinação)",
    "Sem overlap"
]

risco_total_dist = sum(categorias_dist.get(cat, 0) for cat in categorias_risco)
risco_total_rob = sum(categorias_rob.get(cat, 0) for cat in categorias_risco)

percent_risco_dist = (risco_total_dist / len(df_sample)) * 100
percent_risco_rob = (risco_total_rob / len(df_sample)) * 100

print(f"DistilBERT - Exemplos com risco: {risco_total_dist} ({percent_risco_dist:.1f}%)")
print(f"RoBERTa - Exemplos com risco: {risco_total_rob} ({percent_risco_rob:.1f}%)")

if percent_risco_dist > percent_risco_rob:
    print(f"🎯 RoBERTa tem {percent_risco_dist - percent_risco_rob:.1f}pp MENOR risco de alucinação")
else:
    print(f"🎯 DistilBERT tem {percent_risco_rob - percent_risco_dist:.1f}pp MENOR risco de alucinação")

# Análise de respostas explícitas
print("\n✅ ANÁLISE DE RESPOSTAS EXPLÍCITAS:")
print("-"*50)

categorias_explicitas = [
    "Muito Alto (Resposta explícita no contexto)",
    "Alto (Boa correspondência)"
]

explicito_dist = sum(categorias_dist.get(cat, 0) for cat in categorias_explicitas)
explicito_rob = sum(categorias_rob.get(cat, 0) for cat in categorias_explicitas)

percent_explicito_dist = (explicito_dist / len(df_sample)) * 100
percent_explicito_rob = (explicito_rob / len(df_sample)) * 100

print(f"DistilBERT - Respostas explícitas: {explicito_dist} ({percent_explicito_dist:.1f}%)")
print(f"RoBERTa - Respostas explícitas: {explicito_rob} ({percent_explicito_rob:.1f}%)")

# Resumo das médias
print("\n📊 RESUMO DAS MÉDIAS:")
print("-"*50)

media_dist = df_sample['overlap_distilbert'].mean() * 100
media_rob = df_sample['overlap_roberta'].mean() * 100
media_global = (media_dist + media_rob) / 2

print(f"Média de overlap DistilBERT: {media_dist:.1f}%")
print(f"Média de overlap RoBERTa: {media_rob:.1f}%")
print(f"Média global: {media_global:.1f}%")
print(f"Diferença entre modelos: {abs(media_rob - media_dist):.1f}pp")

# Classificação geral baseada na média
print("\n🎯 CLASSIFICAÇÃO GERAL BASEADA NAS MÉDIAS:")
print("-"*50)

def classificar_overlap_geral(media):
    if media >= 80:
        return "Excelente - Respostas predominantemente explícitas"
    elif media >= 60:
        return "Bom - Boa correspondência com contexto"
    elif media >= 40:
        return "Moderado - Algumas inferências necessárias"
    elif media >= 20:
        return "Preocupante - Muitas inferências, risco moderado"
    else:
        return "Crítico - Alto risco de alucinação"

print(f"DistilBERT: {classificar_overlap_geral(media_dist)}")
print(f"RoBERTa: {classificar_overlap_geral(media_rob)}")


📊 CATEGORIZAÇÃO DOS OVERLAPS

📈 DISTRIBUIÇÃO DAS CATEGORIAS DE OVERLAP:
--------------------------------------------------

🤖 DISTILBERT:
  Muito Alto (Resposta explícita no contexto)   :   79 exemplos (  7.9%)
  Sem overlap                                   :  921 exemplos ( 92.1%)

🤖 ROBERTA:
  Muito Alto (Resposta explícita no contexto)   :   79 exemplos (  7.9%)
  Sem overlap                                   :  921 exemplos ( 92.1%)

📊 COMPARAÇÃO ENTRE MODELOS:
--------------------------------------------------
Categoria                                            DistilBERT      RoBERTa    Diferença
--------------------------------------------------------------------------------------
Muito Alto (Resposta explícita no contexto)               7.9%        7.9%        0.0pp
Alto (Boa correspondência)                                0.0%        0.0%        0.0pp
Moderado (Correspondência parcial)                        0.0%        0.0%        0.0pp
Baixo (Possível inferência)         

Com base nos resultados, o valor de 0.079 para o overlap médio, que representa 7.9%, é considerado uma baixa sobreposição. Conforme as instruções do projeto, uma baixa sobreposição pode indicar que a resposta foi inferida incorretamente ou que há potencial para alucinações por parte do modelo. Isso sugere que a maioria das palavras nas respostas não está explicitamente presente no contexto original.

* **D) Avaliação qualitativa por amostragem guiada:**

In [13]:
# ========== TOP 10 EXEMPLOS COM MAIOR SCORE ==========
print("\n" + "="*70)
print("🏆 TOP 10 EXEMPLOS COM MAIOR SCORE")
print("="*70)

# Top 10 globais (maiores scores independente do modelo)
top_10_global = df_sample.nlargest(10, 'melhor_score_global')

for i, (idx, row) in enumerate(top_10_global.iterrows(), 1):
    print(f"\n🏅 RANK {i}:")
    print(f"   ID: {row['_id']}")
    print(f"   Modelo com melhor score: {row['modelo_melhor_score']}")
    print(f"   Score: {row['melhor_score_global']:.4f}")
    print(f"   Pergunta: {row['question'][:80]}..." if len(str(row['question'])) > 80 else f"   Pergunta: {row['question']}")
    print(f"   Contexto: {row['context'][:100]}..." if len(str(row['context'])) > 100 else f"   Contexto: {row['context']}")
    print(f"   DistilBERT: {row['distilbert_score']:.4f}, RoBERTa: {row['roberta_score']:.4f}")

print(f"\n🏆 DISTRIBUIÇÃO NO TOP 10 GLOBAL:")
model_counts = top_10_global['modelo_melhor_score'].value_counts()
for modelo, count in model_counts.items():
    print(f"   {modelo}: {count} exemplos")


🏆 TOP 10 EXEMPLOS COM MAIOR SCORE

🏅 RANK 1:
   ID: <dbpedia:New_Seabury,_Massachusetts>
   Modelo com melhor score: DistilBERT
   Score: 0.9530
   Pergunta: what county is new seabury ma in
   Contexto: New Seabury is a census-designated place (CDP) in the town of Mashpee in Barnstable County, Massachu...
   DistilBERT: 0.9530, RoBERTa: 0.7234

🏅 RANK 2:
   ID: <dbpedia:Beaver_Cove,_Maine>
   Modelo com melhor score: DistilBERT
   Score: 0.9411
   Pergunta: what county is beaver cove me in
   Contexto: Beaver Cove is a town in Piscataquis County, Maine, United States. The population was 122 at the 201...
   DistilBERT: 0.9411, RoBERTa: 0.5576

🏅 RANK 3:
   ID: <dbpedia:Pocasset,_Massachusetts>
   Modelo com melhor score: DistilBERT
   Score: 0.9355
   Pergunta: what county is pomasset ma in
   Contexto: Pocasset is a census-designated place (CDP) in the town of Bourne in Barnstable County, Massachusett...
   DistilBERT: 0.9355, RoBERTa: 0.7514

🏅 RANK 4:
   ID: <dbpedia:Whately,_Mass

In [14]:
# ========== TOP 10 EXEMPLOS COM PIOR SCORE ==========
print("\n" + "="*70)
print("⚠️  TOP 10 EXEMPLOS COM PIOR SCORE")
print("="*70)

# Top 10 piores globais (menores scores independente do modelo)
top_10_piores_global = df_sample.nsmallest(10, 'pior_score_global')

for i, (idx, row) in enumerate(top_10_piores_global.iterrows(), 1):
    print(f"\n🔻 RANK {i} (Pior):")
    print(f"   ID: {row['_id']}")
    print(f"   Modelo com pior score: {row['modelo_pior_score']}")
    print(f"   Pior score: {row['pior_score_global']:.4f}")
    print(f"   Pergunta: {row['question'][:80]}..." if len(str(row['question'])) > 80 else f"   Pergunta: {row['question']}")
    print(f"   Contexto: {row['context'][:100]}..." if len(str(row['context'])) > 100 else f"   Contexto: {row['context']}")
    print(f"   DistilBERT: {row['distilbert_score']:.4f}, RoBERTa: {row['roberta_score']:.4f}")

print(f"\n⚠️  DISTRIBUIÇÃO NO TOP 10 PIORES GLOBAL:")
model_counts_piores = top_10_piores_global['modelo_pior_score'].value_counts()
for modelo, count in model_counts_piores.items():
    print(f"   {modelo}: {count} exemplos")


⚠️  TOP 10 EXEMPLOS COM PIOR SCORE

🔻 RANK 1 (Pior):
   ID: <dbpedia:Arden_on_the_Severn,_Maryland>
   Modelo com pior score: RoBERTa
   Pior score: 0.0004
   Pergunta: where is arden on the sewer located
   Contexto: Arden on the Severn is a census-designated place (CDP) in Anne Arundel County, Maryland, United Stat...
   DistilBERT: 0.1303, RoBERTa: 0.0004

🔻 RANK 2 (Pior):
   ID: <dbpedia:Whately,_Massachusetts>
   Modelo com pior score: RoBERTa
   Pior score: 0.0010
   Pergunta: whately whately
   Contexto: Whately (/ˈweɪtliː/; WAIT-lee) is a town in Franklin County, Massachusetts, United States. The popul...
   DistilBERT: 0.9350, RoBERTa: 0.0010

🔻 RANK 3 (Pior):
   ID: <dbpedia:Discovery-Spring_Garden,_Maryland>
   Modelo com pior score: RoBERTa
   Pior score: 0.0040
   Pergunta: discovery spring garden md
   Contexto: Discovery-Spring Garden is a former census-designated place in Frederick County, Maryland,  United S...
   DistilBERT: 0.0480, RoBERTa: 0.0040

🔻 RANK 4 (Pior):


In [15]:
# ========== EXEMPLOS ALEATÓRIOS COM DISCORDÂNCIA ==========
print("\n" + "="*70)
print("🔄 5 EXEMPLOS ALEATÓRIOS COM DISCORDÂNCIA ENTRE MODELOS")
print("="*70)

# Identificar os índices dos 10 melhores e 10 piores de cada modelo para excluir
top_10_distilbert_idx = df_sample.nlargest(10, 'distilbert_score').index
bottom_10_distilbert_idx = df_sample.nsmallest(10, 'distilbert_score').index
top_10_roberta_idx = df_sample.nlargest(10, 'roberta_score').index
bottom_10_roberta_idx = df_sample.nsmallest(10, 'roberta_score').index

# Unir todos os índices a serem excluídos
excluir_indices = set(top_10_distilbert_idx) | set(bottom_10_distilbert_idx) | set(top_10_roberta_idx) | set(bottom_10_roberta_idx)

# Filtrar o dataframe para excluir os extremos
df_filtrado = df_sample[~df_sample.index.isin(excluir_indices)].copy()

# Verificar se há dados suficientes
if len(df_filtrado) < 5:
    print("❌ Não há exemplos suficientes após excluir os extremos!")
    print(f"   Exemplos disponíveis: {len(df_filtrado)}")
else:
    # Criar uma coluna temporária para verificar se as respostas são diferentes
    def respostas_diferentes(distilbert_answer, roberta_answer):
        # Limpar e comparar as respostas
        distilbert_clean = str(distilbert_answer).strip().lower()
        roberta_clean = str(roberta_answer).strip().lower()
        return distilbert_clean != roberta_clean

    df_filtrado['respostas_diferentes'] = df_filtrado.apply(
        lambda row: respostas_diferentes(row['distilbert_answer'], row['roberta_answer']),
        axis=1
    )

    # Filtrar apenas exemplos onde as respostas são diferentes
    df_diferentes = df_filtrado[df_filtrado['respostas_diferentes']].copy()

    # Verificar se há exemplos com respostas diferentes
    if len(df_diferentes) < 5:
        print(f"❌ Apenas {len(df_diferentes)} exemplos com respostas diferentes fora dos extremos!")
        print("   Mostrando todos os disponíveis:")
        n_mostrar = min(5, len(df_diferentes))
        df_aleatorio = df_diferentes.sample(n=n_mostrar, random_state=42)
    else:
        # Selecionar 5 exemplos aleatórios
        df_aleatorio = df_diferentes.sample(n=5, random_state=42)

    print(f"\n✅ Selecionados {len(df_aleatorio)} exemplos aleatórios com discordância:")
    print(f"   (Excluídos: {len(excluir_indices)} exemplos extremos)")
    print(f"   Disponíveis: {len(df_diferentes)} exemplos com respostas diferentes")
    print("-"*70)

    for i, (idx, row) in enumerate(df_aleatorio.iterrows(), 1):
        print(f"\n📝 EXEMPLO {i}:")
        print(f"   ID: {row['_id']}")
        print(f"   Pergunta: {row['question'][:120]}..." if len(str(row['question'])) > 120 else f"   Pergunta: {row['question']}")
        print(f"   Contexto: {row['context'][:150]}..." if len(str(row['context'])) > 150 else f"   Contexto: {row['context']}")
        print(f"\n   🤖 DISTILBERT:")
        print(f"      Resposta: {row['distilbert_answer']}")
        print(f"      Score: {row['distilbert_score']:.4f}")
        print(f"      Overlap: {row['overlap_distilbert']:.2%}")
        print(f"\n   🤖 ROBERTA:")
        print(f"      Resposta: {row['roberta_answer']}")
        print(f"      Score: {row['roberta_score']:.4f}")
        print(f"      Overlap: {row['overlap_roberta']:.2%}")
        print(f"\n   📊 COMPARAÇÃO:")
        print(f"      Diferença de score: {row['score_difference']:.4f}")
        print(f"      Diferença de overlap: {row['overlap_difference']:.2%}")
        print(f"      Melhor por score: {row['melhor_modelo_score']}")
        print(f"      Melhor por overlap: {row['melhor_modelo_overlap']}")

    # Estatísticas dos exemplos selecionados
    print("\n" + "="*70)
    print("📊 ESTATÍSTICAS DOS EXEMPLOS SELECIONADOS")
    print("="*70)

    print(f"\n📈 MÉDIAS DOS {len(df_aleatorio)} EXEMPLOS:")
    print(f"   Score DistilBERT: {df_aleatorio['distilbert_score'].mean():.4f}")
    print(f"   Score RoBERTa: {df_aleatorio['roberta_score'].mean():.4f}")
    print(f"   Overlap DistilBERT: {df_aleatorio['overlap_distilbert'].mean():.2%}")
    print(f"   Overlap RoBERTa: {df_aleatorio['overlap_roberta'].mean():.2%}")

    print(f"\n🏆 DISTRIBUIÇÃO DE MELHORES:")
    print(f"   Por score: {df_aleatorio['melhor_modelo_score'].value_counts().to_dict()}")
    print(f"   Por overlap: {df_aleatorio['melhor_modelo_overlap'].value_counts().to_dict()}")

    print(f"\n📏 COMPRIMENTO MÉDIO:")
    print(f"   Pergunta: {df_aleatorio['question_length'].mean():.1f} palavras")
    print(f"   Contexto: {df_aleatorio['context_length'].mean():.1f} palavras")


🔄 5 EXEMPLOS ALEATÓRIOS COM DISCORDÂNCIA ENTRE MODELOS

✅ Selecionados 5 exemplos aleatórios com discordância:
   (Excluídos: 38 exemplos extremos)
   Disponíveis: 285 exemplos com respostas diferentes
----------------------------------------------------------------------

📝 EXEMPLO 1:
   ID: <dbpedia:Brunswick_Station,_Maine>
   Pergunta: where is brunswick station on map
   Contexto: Brunswick Station is a census-designated place (CDP) within the town of Brunswick in Cumberland County, Maine, United States. The population was 578 a...

   🤖 DISTILBERT:
      Resposta: Cumberland County, Maine, United States
      Score: 0.3700
      Overlap: 0.00%

   🤖 ROBERTA:
      Resposta: Cumberland County, Maine
      Score: 0.1254
      Overlap: 0.00%

   📊 COMPARAÇÃO:
      Diferença de score: -0.2445
      Diferença de overlap: 0.00%
      Melhor por score: DistilBERT
      Melhor por overlap: Empate

📝 EXEMPLO 2:
   ID: <dbpedia:Montague,_Massachusetts>
   Pergunta: where is montgomery ma

Eu escolheria o modelo DistilBERT para integração em produção, fundamentando minha decisão na análise qualitativa e quantitativa dos resultados:

**1. CONSISTÊNCIA SUPERIOR EM CASOS CRÍTICOS:**

- Nos 10 melhores exemplos, DistilBERT domina com 8 dos 10 casos (80%), demonstrando excelente desempenho em perguntas bem formuladas.

- Nos 10 piores exemplos, RoBERTa está presente em todos os 10 casos (100%), mostrando vulnerabilidade extrema em situações desafiadoras.

- O exemplo mais revelador é o ID <dbpedia:Whately,_Massachusetts>: aparece tanto nos melhores (Rank 4 com DistilBERT: 0.9350) quanto nos piores (Rank 2 com RoBERTa: 0.0010) para a mesma pergunta "whately whately". Essa inconsistência extrema do RoBERTa é inaceitável para produção.

**2. ROBUSTEZ A PERGUNTAS MAL FORMULADAS:**

- Nos piores casos, RoBERTa falha catastróficamente com scores próximos a zero (0.0004, 0.0010, 0.0040), enquanto DistilBERT mantém desempenho razoável mesmo nestes cenários.

- Exemplo: Para "where is arden on the sewer located" (pergunta com erro de digitação "sewer" em vez de "Severn"), DistilBERT: 0.1303 vs RoBERTa: 0.0004.

**3. RESILIÊNCIA OPERACIONAL:**

- DistilBERT mostra menor variabilidade de desempenho, um fator crítico para sistemas em produção que precisam de comportamento previsível.

- A análise dos 5 exemplos aleatórios com discordância revela que DistilBERT tem melhor score médio (0.3580 vs 0.2840) quando os modelos divergem.

**4. CARACTERÍSTICAS TÉCNICAS FAVORÁVEIS PARA PRODUÇÃO:**

- Menor consumo computacional: DistilBERT é uma versão distilada, mais leve e rápida que RoBERTa, resultando em:

- Menor latência (crítico para aplicações em tempo real)

- Menor custo de infraestrutura

- Maior capacidade de escalabilidade

- Compatibilidade com restrições de memória em ambientes com recursos limitados.

**5. ANÁLISE DE RISCO:**

- RoBERTa demonstra falhas catastróficas frequentes (múltiplos scores < 0.05), criando risco operacional inaceitável.

- DistilBERT oferece degradação mais gradual do desempenho, permitindo mecanismos de fallback e tratamento de erros mais eficazes.

**6. CASOS DE USO PRÁTICOS:**

- Para aplicações de geolocalização (maioria dos exemplos analisados), DistilBERT mostra melhor extração de informações estruturadas (nomes de condados, estados).

- A capacidade de manter performance razoável mesmo com pequenos erros nas perguntas é essencial para sistemas que lidam com entradas de usuários reais.